In [ ]:
from webdriver_manager.chrome import ChromeDriverManager   #-------
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from datetime import datetime
import json
from dateutil.relativedelta import *
import os

def scrolling_down(t):
    for i in range(t):
        driver.execute_script("window.scrollBy(0,2000)","")
        time.sleep(0.5)
        
def get_text(x):
    return driver.find_element(By.XPATH,x).text

def get_herf(x):
    return driver.find_element(By.XPATH,x).get_attribute("href")

def click(x):
    driver.find_element(By.XPATH,x).click()

# gen links

In [ ]:
# driver = webdriver.Chrome()
driver = webdriver.Chrome(ChromeDriverManager(version='108.0.5359.71').install())   #-----------------


In [ ]:
page = 1
driver.get(f'https://th.zmyhome.com/buy?page={page}&sortFilter=ads&per-page=40')
driver.find_element(By.XPATH,'/html/body/div[2]/button').click()
max_page = driver.find_element(By.XPATH,'/html/body/div[1]/section[2]/div/div/div/div[5]/ul/li[8]/a').text
max_page = int(max_page)
max_page

In [ ]:
def get_links(page):
    driver.get(f'https://th.zmyhome.com/buy?page={page}&sortFilter=ads&per-page=40')
    time.sleep(3)
    LINK = []
    c = 1
    for i in range(50):
        try:
            p = f'/html/body/div[1]/section[2]/div/div/div/div[4]/div[{i}]/article/figure/a'
            link = driver.find_element(By.XPATH,p).get_attribute("href")
#             print(page,c,i,link)
            LINK.append(link)
            c += 1
        except:
            pass
    return LINK
    
# max_page = 1 
# page = 1
# LINKS = []
# for page in range(1,max_page+1):
#     LINK = get_links(page)
#     LINKS += LINK

In [ ]:
LINK

In [ ]:
len(LINKS)

# get details

In [ ]:
# driver.get(LINKS[1])

def get_Details(url):

#     driver.get('https://th.zmyhome.com/property/H362887')
    driver.get(url)
    time.sleep(3)

    def get_detailbox():
        t = get_text('/html/body/div[1]/section[1]/div/div/div/div[3]/div[2]').split('\n')[1:]
        d = {}
        for i in range(len(t)):
            if i%2==0:
                d[t[i]] = t[i+1]
        return d

    def get_near():
        t = get_text('/html/body/div[1]/section[1]/div/div/div/div[3]/div[5]').split('\n')
        location = t[2]
        t = [x for x in t if x not in ['ทำเลที่ตั้ง','คลิกดูแผนที่','น้อยลง']]
        idx = None
        near = []
        for i in t:
            if '(' not in i:
                idx = i
            else:
    #             print(idx,i)
                near.append(f'{idx}-{i}')
        return near,location
    
    def create_date(d):
        dd = d.split()
        if 'เดือน' in d:
            date = datetime.now() + relativedelta(months=-1*int(dd[1]))
        elif 'วัน' in d:
            date = datetime.now() + relativedelta(days=-1*int(dd[1]))
        elif 'ปี' in d:
            date = datetime.now() + relativedelta(years=-1*int(dd[1]))
        else:
            date = None
        return date.strftime("%Y%m%d")

    d = get_detailbox()
    try:
        click('/html/body/div[1]/section[1]/div/div/div/div[3]/div[5]/a')  #attend ทำเลที่ตั้ง
    except:
        pass
    d['header'] = get_text('/html/body/div[1]/section[1]/div/div/div/div[1]/h2')
    d['gps'] = get_herf('/html/body/div[1]/section[1]/div/div/div/div[3]/div[6]/div/a')
    d['price_per_area'] = get_text('/html/body/div[1]/section[1]/div/div/div/div[3]/div[1]/ul/li/small')
    d['seen'] = get_text('/html/body/div[1]/section[1]/div/div/div/div[2]/div[2]/div/ul[2]/li[1]/a/span[2]')
    d['love'] = get_text('/html/body/div[1]/section[1]/div/div/div/div[2]/div[2]/div/ul[2]/li[2]/a/span[2]')
    d['last_update'] = create_date(get_text('/html/body/div[1]/section[1]/div/div/div/div[2]/div[2]/div/ul[2]/li[4]/a/span[2]'))
    d['near'],d['location'] = get_near()
    d['timestamp'] = time.time()
    return d

# url = 'https://th.zmyhome.com/property/H362887'
# get_Details(url)





In [ ]:
#test
driver.get('https://th.zmyhome.com/property/V138493')
# time.sleep(3)

In [ ]:
# get_herf('/html/body/div[1]/section[1]/div/div/div/div[3]/div[6]/div/a')
driver.get('https://th.zmyhome.com/property/H18071')

In [ ]:
get_text('/html/body/div[1]/section[1]/div/div/div/div[1]/div')

In [ ]:
max_page = 2
page = 1

if not os.path.exists('data'):
  os.makedirs('data')

for page in range(1,max_page+1):
    LINK = get_links(page)
#     LINK = LINK[:5]  #---------------
    
    try:
        with open('data/zmy_data.json', 'r') as openfile:
            D = json.load(openfile)
    except:
        D = {}

    for url in LINK:
        print(url)
        d = get_Details(url)
        print(d)
        D[url] = d

    with open("data/zmy_data.json", "w") as outfile:
        outfile.write(json.dumps(D, indent=4))

In [ ]:
'https://th.zmyhome.com/property/V179876' in D.keys()

In [6]:
import json

with open('data/zmy_data.json', 'r') as openfile:
    D = json.load(openfile)
D['https://th.zmyhome.com/property/V180552']['gpss']

KeyError: 'gpss'

In [ ]:
url = 'https://th.zmyhome.com/property/V179876'
D[url]['last_update'] >

In [ ]:
datetime.now().strftime("%Y%m%d")

In [ ]:
def diff_month(d2,d1):
#     d2 = datetime.now().strftime("%Y%m%d")
    y = int(d2[:4])-int(d1[:4])
    m = int(d2[4:6])-int(d1[4:6])
    return 12*y+m
    

d1 = '20221008'
d2 = '20221105'
diff_month(d2,d1)


# y = int(d2[:4])-int(d1[:4])
# m = int(d2[4:6])-int(d1[4:6])
# d = int(d2[6:])-int(d1[6:])
# y,m,d
# # int(d2[:3])-int('202212')

In [ ]:
dm = 12*y+m
dm

In [ ]:
def create_date(dd):
    if not dd:
        return None
    d = dd.split()
    if 'ชม.' in d:
        date = str(datetime.datetime.now() + datetime.timedelta(hours=-1*int(d[0])))
    elif 'วัน' in d:
        date = str(datetime.datetime.now() + datetime.timedelta(days=-1*int(d[0])))
    elif 'เวลา' in d:
        if len(d[0]) == 1: d[0] = '0'+d[0]
        date = f'{datetime.datetime.now().strftime("%Y")}-{month[d[1]]}-{d[0]} {d[3]}:00:000000'
    elif d[-1] in month:
        if len(d[0]) == 1: d[0] = '0'+d[0]
        date = f'{datetime.datetime.now().strftime("%Y")}-{month[d[1]]}-{d[0]} {str(datetime.datetime.now()).split()[-1]}'
    else:
        date = None
    return date



In [ ]:
dates = ['อัพเดตล่าสุด 8 เดือน','อัพเดตล่าสุด 8 วัน','อัพเดตล่าสุด 8 ปี']

def create_date(d):
    dd = d.split()
    if 'เดือน' in d:
        date = datetime.now() + relativedelta(months=-1*int(dd[1]))
    elif 'วัน' in d:
        date = datetime.now() + relativedelta(days=-1*int(dd[1]))
    elif 'ปี' in d:
        date = datetime.now() + relativedelta(years=-1*int(dd[1]))
    else:
        date = None
    return date.strftime("%Y%m%d")

create_date(dates[0])

In [ ]:
dates[0].split()

In [ ]:
datetime.datetime.now() + datetime.timedelta(months=1)

In [ ]:
from datetime import datetime, timedelta
from dateutil.relativedelta import *

date = datetime.now()
print(date)
# 2018-09-24 13:24:04.007620

date = date + relativedelta(months=+6)
print(date)

In [ ]:
import time

# current timestamp
x = time.time()
x

In [ ]:
# Serializing json
json_object = json.dumps(D, indent=5)
 
# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
json_object = json.dumps(D, indent = 4) 
print(json_object)

In [ ]:
click('/html/body/div[1]/section[1]/div/div/div/div[3]/div[5]/a')  #attend ทำเลที่ตั้ง

In [ ]:
d['gps'] = get_herf('/html/body/div[1]/section[1]/div/div/div/div[3]/div[5]/div/a')
d['price_per_area'] = get_text('/html/body/div[1]/section[1]/div/div/div/div[3]/div[1]/ul/li/small')
d['seen'] = get_text('/html/body/div[1]/section[1]/div/div/div/div[2]/div[2]/div/ul[2]/li[1]/a/span[2]')
d['love'] = get_text('/html/body/div[1]/section[1]/div/div/div/div[2]/div[2]/div/ul[2]/li[2]/a/span[2]')
d['last_update'] = get_text('/html/body/div[1]/section[1]/div/div/div/div[2]/div[2]/div/ul[2]/li[4]/a/span[2]')
d

In [ ]:
t = get_text('/html/body/div[1]/section[1]/div/div/div/div[3]/div[5]').split('\n')
location = t[2]
t = [x for x in t if x not in ['ทำเลที่ตั้ง','คลิกดูแผนที่','น้อยลง']]
idx = None
near = []
for i in t:
    if '(' not in i:
        idx = i
    else:
        print(idx,i)
        near.append(f'{idx}-{i}')
near

In [ ]:

# Getting the current date and time
dt = datetime.now()
dt

In [ ]:
gps = get_herf('/html/body/div[1]/section[1]/div/div/div/div[3]/div[5]/div/a')
gps

In [ ]:
price_per_area = get_text('/html/body/div[1]/section[1]/div/div/div/div[3]/div[1]/ul/li/small')
price_per_area

In [ ]:
seen = get_text('/html/body/div[1]/section[1]/div/div/div/div[2]/div[2]/div/ul[2]/li[1]/a/span[2]')
seen
love = get_text('/html/body/div[1]/section[1]/div/div/div/div[2]/div[2]/div/ul[2]/li[2]/a/span[2]')
love
last_update = get_text('/html/body/div[1]/section[1]/div/div/div/div[2]/div[2]/div/ul[2]/li[4]/a/span[2]')
last_update

In [ ]:
t = get_text('/html/body/div[1]/section[1]/div/div/div/div[3]/div[5]').split('\n')
t

In [ ]:
location = t[2]
t = [x for x in t if x not in ['ทำเลที่ตั้ง','คลิกดูแผนที่','น้อยลง']]
idx = None
near = []
for i in t:
    if '(' not in i:
        idx = i
    else:
        print(idx,i)
        near.append()
        

In [ ]:
t = get_text('/html/body/div[1]/section[1]/div/div/div/div[3]/div[2]')
t

In [ ]:
d = {}
t = t.split('\n')[1:]

len(t)

In [ ]:
d = {}
for i in range(len(t)//2):
    print(i)
    if i%2==0:
        d[t[i]] = t[i+1]
d

In [ ]:
for i in range(1,5):
    print(i)

In [ ]:
def scrolling_down(t):
    for i in range(t):
        driver.execute_script("window.scrollBy(0,2000)","")
        time.sleep(0.5)